> https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqu7n114zx


El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso, descripcion, estado_caso, categoria, latitud, longitud).  

Los  posibles estados  que puede tener  un caso son 1: caso abierto, 2: caso  resuelto, 3: cerrado sin resolución.  

Las fechas se encuentran en el formato YYYY-MM-DD. Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para 
pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respu
esta positiva (1) o negativa (0) de parte de él.

El  sector  encargado  de  las  estadísticas  oficiales  del  GCPD  quiere  con  esta  información  analizar  las  siguientes situaciones: 

- Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman). 

- Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la batiseñal, participó en la resolución). 

- Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos. 
    
---

In [1]:
# Set-up y vista rápida de las dos bases de datos truchas
import pandas as pd

df_gcpd = pd.read_csv('data/2018C1_GCPD.csv')
display(df_gcpd)

df_gordon = pd.read_csv('data/2018C1_gordon.csv')
display(df_gordon)

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud
0,2018-05-03,1,Mataron a uno,1,Arseny,100,100.0
1,2018-05-03,2,Mataron al Guason!!,2,Robo,100,100.0
2,2018-05-03,3,Incendio,1,Robo,100,100.0
3,2018-05-03,4,Robaron un banco,3,Incendio,100,100.0
4,2018-05-03,5,Mataron a uno,2,Incendio,100,100.0
5,2018-05-03,6,Mataron a uno,1,Arseny,100,100.0
6,2018-05-03,7,Mataron a uno,2,Robo,100,100.0
7,2018-05-03,8,Mataron a uno,3,Incendio100,100,NaN
8,2018-05-03,9,Mataron a uno,2,Arseny,100,100.0
9,2018-05-03,10,Mataron a uno,1,Robo,100,100.0


,id_caso,respuesta
0,1,1
1,2,0
2,3,0
3,4,0
4,5,1
5,6,0
6,7,1
7,8,0
8,9,1
9,10,0


Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman).